In [1]:
import numpy as np

import glob
import re
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import scipy.stats.stats as stats

#from sklearn.decomposition import PCA

In [2]:
import zipfile

# read the dataset using the compression zip
all_data = pd.read_csv('all_data.zip',compression='zip')

In [3]:
all_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url
0,12027_rev1_d,NaN,NaN,NaN,NaN,NaN,ABANDONED,Emerson Nolan,1,http://review.couchbase.org/#/c/12027/1
1,12050_rev1_d,NaN,NaN,NaN,NaN,NaN,MERGED,Emerson Nolan,1,http://review.couchbase.org/#/c/12050/1
2,12050_rev2_d,Uploaded patch set 2.,Jeffrey Roth,2012-01-04 21:45:42.382000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,MERGED,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
3,12050_rev2_d,Patch Set 2: VerifiedMatt/Dustin/Mike:I have r...,Jeffrey Roth,2012-01-04 21:51:47.408000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,MERGED,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
4,12050_rev2_d,Patch Set 2: I would prefer that you didn't su...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,MERGED,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
...,...,...,...,...,...,...,...,...,...,...
422615,85948_rev2_d,Change has been successfully cherry-picked as ...,Emely Wilson,2017-11-23 10:51:00.946000000,NaN,CBQE-4333: Fix the data population issues,MERGED,Emely Wilson,2,http://review.couchbase.org/#/c/85948/2
422616,85957_rev1_d,Uploaded patch set 1.,Emely Wilson,2017-11-23 16:01:28.983000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,MERGED,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1
422617,85957_rev1_d,Patch Set 1: Code-Review+2,Alissa Huang,2017-11-23 16:04:40.102000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,MERGED,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1
422618,85957_rev1_d,Patch Set 1: Verified+1,Emely Wilson,2017-11-23 16:05:06.479000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,MERGED,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1


Convert Status to 0 if abandoned and 1 if merged

In [4]:
all_data['status'] = np.where((all_data['status'] == 'MERGED') ,True,False)*1

In [5]:
all_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url
0,12027_rev1_d,NaN,NaN,NaN,NaN,NaN,0,Emerson Nolan,1,http://review.couchbase.org/#/c/12027/1
1,12050_rev1_d,NaN,NaN,NaN,NaN,NaN,1,Emerson Nolan,1,http://review.couchbase.org/#/c/12050/1
2,12050_rev2_d,Uploaded patch set 2.,Jeffrey Roth,2012-01-04 21:45:42.382000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
3,12050_rev2_d,Patch Set 2: VerifiedMatt/Dustin/Mike:I have r...,Jeffrey Roth,2012-01-04 21:51:47.408000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
4,12050_rev2_d,Patch Set 2: I would prefer that you didn't su...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
...,...,...,...,...,...,...,...,...,...,...
422615,85948_rev2_d,Change has been successfully cherry-picked as ...,Emely Wilson,2017-11-23 10:51:00.946000000,NaN,CBQE-4333: Fix the data population issues,1,Emely Wilson,2,http://review.couchbase.org/#/c/85948/2
422616,85957_rev1_d,Uploaded patch set 1.,Emely Wilson,2017-11-23 16:01:28.983000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,1,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1
422617,85957_rev1_d,Patch Set 1: Code-Review+2,Alissa Huang,2017-11-23 16:04:40.102000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,1,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1
422618,85957_rev1_d,Patch Set 1: Verified+1,Emely Wilson,2017-11-23 16:05:06.479000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,1,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1


In [6]:
all_data['PR_number'].isna().sum()

0

In [7]:
all_data.isna().sum()

PR_number               0
Comment             13819
Author              13838
Date                13836
Commit_message     164978
Descriptions        13866
status                  0
main_author             5
revision_number         0
url                     0
dtype: int64

Drop all rows with nan comments, Author and Dates

In [8]:
all_data = all_data.dropna(subset=['Comment'])
all_data = all_data.dropna(subset=['Author'])
all_data = all_data.dropna(subset=['Date'])
all_data = all_data.dropna(subset=['main_author'])

In [9]:
all_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url
2,12050_rev2_d,Uploaded patch set 2.,Jeffrey Roth,2012-01-04 21:45:42.382000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
3,12050_rev2_d,Patch Set 2: VerifiedMatt/Dustin/Mike:I have r...,Jeffrey Roth,2012-01-04 21:51:47.408000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
4,12050_rev2_d,Patch Set 2: I would prefer that you didn't su...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
5,12050_rev2_d,Patch Set 2:You also need to fix the commit me...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
6,12050_rev2_d,Patch Set 2:I don't know that changing the pac...,Andreas Short,2012-01-05 00:24:39.162000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2
...,...,...,...,...,...,...,...,...,...,...
422615,85948_rev2_d,Change has been successfully cherry-picked as ...,Emely Wilson,2017-11-23 10:51:00.946000000,NaN,CBQE-4333: Fix the data population issues,1,Emely Wilson,2,http://review.couchbase.org/#/c/85948/2
422616,85957_rev1_d,Uploaded patch set 1.,Emely Wilson,2017-11-23 16:01:28.983000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,1,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1
422617,85957_rev1_d,Patch Set 1: Code-Review+2,Alissa Huang,2017-11-23 16:04:40.102000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,1,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1
422618,85957_rev1_d,Patch Set 1: Verified+1,Emely Wilson,2017-11-23 16:05:06.479000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,1,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1


Add a column with only the Pr-number without the patch version

In [10]:
all_data['PR_number_no_patch_number'] = all_data['PR_number'].str[:5]

In [11]:
all_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number
2,12050_rev2_d,Uploaded patch set 2.,Jeffrey Roth,2012-01-04 21:45:42.382000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
3,12050_rev2_d,Patch Set 2: VerifiedMatt/Dustin/Mike:I have r...,Jeffrey Roth,2012-01-04 21:51:47.408000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
4,12050_rev2_d,Patch Set 2: I would prefer that you didn't su...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
5,12050_rev2_d,Patch Set 2:You also need to fix the commit me...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
6,12050_rev2_d,Patch Set 2:I don't know that changing the pac...,Andreas Short,2012-01-05 00:24:39.162000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
...,...,...,...,...,...,...,...,...,...,...,...
422615,85948_rev2_d,Change has been successfully cherry-picked as ...,Emely Wilson,2017-11-23 10:51:00.946000000,NaN,CBQE-4333: Fix the data population issues,1,Emely Wilson,2,http://review.couchbase.org/#/c/85948/2,85948
422616,85957_rev1_d,Uploaded patch set 1.,Emely Wilson,2017-11-23 16:01:28.983000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,1,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1,85957
422617,85957_rev1_d,Patch Set 1: Code-Review+2,Alissa Huang,2017-11-23 16:04:40.102000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,1,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1,85957
422618,85957_rev1_d,Patch Set 1: Verified+1,Emely Wilson,2017-11-23 16:05:06.479000000,NaN,CBQE-4333: make eventing dataset tc's work fro...,1,Emely Wilson,1,http://review.couchbase.org/#/c/85957/1,85957


Deleting all the comments from bots.

In [12]:
bot_comments = ['Uploaded patch set','Added to reviewer','Verified','Change has been successfully','Your change could not be merged due to a path conflict.','Code-Review','(1 comment)'
                ,'Logs','log','Cherry Picked', 'was rebased','Cherry Picked from branch master.','cherry picked',
                'Cherry Picked','comments','Published edit on patch set']
bot_comments_pattern = '|'.join(bot_comments)


In [13]:
all_data.Comment.str.contains(bot_comments_pattern).sum()

C:\Users\GNR\AppData\Local\Temp\ipykernel_588\1003850507.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  all_data.Comment.str.contains(bot_comments_pattern).sum()


287332

We have 287332 comments from bots

And now we are going to delete them.

In [14]:
all_data = all_data[all_data.Comment.str.contains(bot_comments_pattern) == False]

C:\Users\GNR\AppData\Local\Temp\ipykernel_588\3367746845.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  all_data = all_data[all_data.Comment.str.contains(bot_comments_pattern) == False]


In [15]:
all_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number
4,12050_rev2_d,Patch Set 2: I would prefer that you didn't su...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
5,12050_rev2_d,Patch Set 2:You also need to fix the commit me...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
6,12050_rev2_d,Patch Set 2:I don't know that changing the pac...,Andreas Short,2012-01-05 00:24:39.162000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
7,12050_rev2_d,Patch Set 2:I don't know that changing the pac...,Andreas Short,2012-01-05 00:24:39.162000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
8,12050_rev2_d,Patch Set 2: No scoreThe only reason I brought...,Kenny Maynard,2012-01-05 18:45:06.860000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
...,...,...,...,...,...,...,...,...,...,...,...
422502,85713_rev2_d,Created a revert of this change as I4c00a1f817...,Alissa Huang,2017-11-19 04:56:20.273000000,NaN,CBQE-4388 set ulimit to 102400 in centos serve...,1,Frederick Booker,2,http://review.couchbase.org/#/c/85713/2,85713
422544,85827_rev3_d,"MISMATCHED INLINE COMMENTLine:44, pytests/even...",Emely Wilson,2017-11-21 11:03:55.013000000,NaN,CBQE-4333: Add eventing negative test cases,1,Emely Wilson,3,http://review.couchbase.org/#/c/85827/3,85827
422560,85859_rev1_d,Assignee added: Nelson Emerson Dukekson <Nelso...,Jakobe Johnson,2017-11-21 21:26:16.466000000,Fix one problem with STEP 1.,CBQE-4390:Fix one problem with STEP 1.,1,Jakobe Johnson,1,http://review.couchbase.org/#/c/85859/1,85859
422561,85859_rev1_d,Assignee deleted: Nelson Emerson Dukekson <Nel...,Jakobe Johnson,2017-11-21 21:27:16.130000000,Fix one problem with STEP 1.,CBQE-4390:Fix one problem with STEP 1.,1,Jakobe Johnson,1,http://review.couchbase.org/#/c/85859/1,85859


I noticed that only the comments that start with 'Patch Set {number of the patch} are useful for our analysis so I am going to keep only those comments.

In [16]:
useful_data = all_data[all_data['Comment'].str[:9].str.contains('Patch Set') == True]

In [17]:
useful_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number
4,12050_rev2_d,Patch Set 2: I would prefer that you didn't su...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
5,12050_rev2_d,Patch Set 2:You also need to fix the commit me...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
6,12050_rev2_d,Patch Set 2:I don't know that changing the pac...,Andreas Short,2012-01-05 00:24:39.162000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
7,12050_rev2_d,Patch Set 2:I don't know that changing the pac...,Andreas Short,2012-01-05 00:24:39.162000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
8,12050_rev2_d,Patch Set 2: No scoreThe only reason I brought...,Kenny Maynard,2012-01-05 18:45:06.860000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050
...,...,...,...,...,...,...,...,...,...,...,...
420134,80604_rev2_d,Patch Set 2:Got it - will remember that.,Ellie Kidd,2017-07-13 18:41:21.038000000,NaN,CBQE-3067: Use better FTS level compaction con...,1,Ellie Kidd,2,http://review.couchbase.org/#/c/80604/2,80604
420185,80992_rev1_d,Patch Set 1:MemoryQuota may set in param with ...,Frederick Booker,2017-07-18 15:43:30.472000000,NaN,CBQE-4209: Added the cbas installation and ser...,1,Alfred Park,1,http://review.couchbase.org/#/c/80992/1,80992
420805,82094_rev1_d,Patch Set 1:This change has already been revie...,Monique Barker,2017-08-09 02:25:02.503000000,NaN,Adding subquery tests to watson branch.,1,Dylan Paul,1,http://review.couchbase.org/#/c/82094/1,82094
421737,84162_rev1_d,Patch Set 1:test suite lww in entbackup restor...,Frederick Booker,2017-10-09 19:52:13.747000000,NaN,CBQE-1898 lww tests are covered in py-entbacku...,1,Frederick Booker,1,http://review.couchbase.org/#/c/84162/1,84162


Now we are going to remove the Patch set number from every comment.

In [18]:
useful_data['Comment'] = all_data['Comment'].str[12:]

C:\Users\GNR\AppData\Local\Temp\ipykernel_588\13972313.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_data['Comment'] = all_data['Comment'].str[12:]


In [19]:
bot_comments = ['Patch Set','patch set']
bot_comments_pattern = '|'.join(bot_comments)
all_data = all_data[all_data.Comment.str.contains(bot_comments_pattern) == False]
all_data.Comment.str.contains(bot_comments_pattern).sum()

0

Find the length of every comment(METRICS)

In [20]:
useful_data['comment_length'] = useful_data['Comment'].str.len()

C:\Users\GNR\AppData\Local\Temp\ipykernel_588\2463225354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_data['comment_length'] = useful_data['Comment'].str.len()


Removing all the comments with more than 500 characters.(METRICS)


In [21]:
useful_data = useful_data[useful_data['comment_length'] <= 500]

In [22]:
useful_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number,comment_length
4,12050_rev2_d,I would prefer that you didn't submit thisI w...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,318
5,12050_rev2_d,You also need to fix the commit message so it ...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,77
9,12050_rev2_d,I would prefer that you didn't submit this-1 ...,Kenny Maynard,2012-01-05 22:42:21.672000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,76
10,12050_rev2_d,"I don't have a huge opinion on it, really. My...",Noah Dean,2012-01-06 03:50:33.697000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,388
12,12050_rev3_d,"I think Rags is working on this today too, so ...",Andreas Short,2012-01-08 21:36:57.811000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,3,http://review.couchbase.org/#/c/12050/3,12050,161
...,...,...,...,...,...,...,...,...,...,...,...,...
420134,80604_rev2_d,Got it - will remember that.,Ellie Kidd,2017-07-13 18:41:21.038000000,NaN,CBQE-3067: Use better FTS level compaction con...,1,Ellie Kidd,2,http://review.couchbase.org/#/c/80604/2,80604,28
420185,80992_rev1_d,MemoryQuota may set in param with default valu...,Frederick Booker,2017-07-18 15:43:30.472000000,NaN,CBQE-4209: Added the cbas installation and ser...,1,Alfred Park,1,http://review.couchbase.org/#/c/80992/1,80992,113
420805,82094_rev1_d,This change has already been reviewed by dev f...,Monique Barker,2017-08-09 02:25:02.503000000,NaN,Adding subquery tests to watson branch.,1,Dylan Paul,1,http://review.couchbase.org/#/c/82094/1,82094,95
421737,84162_rev1_d,test suite lww in entbackup restore is also re...,Frederick Booker,2017-10-09 19:52:13.747000000,NaN,CBQE-1898 lww tests are covered in py-entbacku...,1,Frederick Booker,1,http://review.couchbase.org/#/c/84162/1,84162,61


In [23]:
useful_data['comment_length'] = useful_data['Comment'].str.len()

C:\Users\GNR\AppData\Local\Temp\ipykernel_588\2463225354.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_data['comment_length'] = useful_data['Comment'].str.len()


In [24]:
useful_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number,comment_length
4,12050_rev2_d,I would prefer that you didn't submit thisI w...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,318
5,12050_rev2_d,You also need to fix the commit message so it ...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,77
9,12050_rev2_d,I would prefer that you didn't submit this-1 ...,Kenny Maynard,2012-01-05 22:42:21.672000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,76
10,12050_rev2_d,"I don't have a huge opinion on it, really. My...",Noah Dean,2012-01-06 03:50:33.697000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,388
12,12050_rev3_d,"I think Rags is working on this today too, so ...",Andreas Short,2012-01-08 21:36:57.811000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,3,http://review.couchbase.org/#/c/12050/3,12050,161
...,...,...,...,...,...,...,...,...,...,...,...,...
420134,80604_rev2_d,Got it - will remember that.,Ellie Kidd,2017-07-13 18:41:21.038000000,NaN,CBQE-3067: Use better FTS level compaction con...,1,Ellie Kidd,2,http://review.couchbase.org/#/c/80604/2,80604,28
420185,80992_rev1_d,MemoryQuota may set in param with default valu...,Frederick Booker,2017-07-18 15:43:30.472000000,NaN,CBQE-4209: Added the cbas installation and ser...,1,Alfred Park,1,http://review.couchbase.org/#/c/80992/1,80992,113
420805,82094_rev1_d,This change has already been reviewed by dev f...,Monique Barker,2017-08-09 02:25:02.503000000,NaN,Adding subquery tests to watson branch.,1,Dylan Paul,1,http://review.couchbase.org/#/c/82094/1,82094,95
421737,84162_rev1_d,test suite lww in entbackup restore is also re...,Frederick Booker,2017-10-09 19:52:13.747000000,NaN,CBQE-1898 lww tests are covered in py-entbacku...,1,Frederick Booker,1,http://review.couchbase.org/#/c/84162/1,84162,61


Deleting again some bots comments.

In [25]:
bot_comments = ['Patch Set','hudson','cv','jasmin', 'Failed','Started', 'http', 'https']
bot_comments_pattern = '|'.join(bot_comments)
useful_data = useful_data[useful_data.Comment.str.contains(bot_comments_pattern) == False]

In [26]:
useful_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number,comment_length
4,12050_rev2_d,I would prefer that you didn't submit thisI w...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,318
5,12050_rev2_d,You also need to fix the commit message so it ...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,77
9,12050_rev2_d,I would prefer that you didn't submit this-1 ...,Kenny Maynard,2012-01-05 22:42:21.672000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,76
10,12050_rev2_d,"I don't have a huge opinion on it, really. My...",Noah Dean,2012-01-06 03:50:33.697000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,388
12,12050_rev3_d,"I think Rags is working on this today too, so ...",Andreas Short,2012-01-08 21:36:57.811000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,3,http://review.couchbase.org/#/c/12050/3,12050,161
...,...,...,...,...,...,...,...,...,...,...,...,...
420134,80604_rev2_d,Got it - will remember that.,Ellie Kidd,2017-07-13 18:41:21.038000000,NaN,CBQE-3067: Use better FTS level compaction con...,1,Ellie Kidd,2,http://review.couchbase.org/#/c/80604/2,80604,28
420185,80992_rev1_d,MemoryQuota may set in param with default valu...,Frederick Booker,2017-07-18 15:43:30.472000000,NaN,CBQE-4209: Added the cbas installation and ser...,1,Alfred Park,1,http://review.couchbase.org/#/c/80992/1,80992,113
420805,82094_rev1_d,This change has already been reviewed by dev f...,Monique Barker,2017-08-09 02:25:02.503000000,NaN,Adding subquery tests to watson branch.,1,Dylan Paul,1,http://review.couchbase.org/#/c/82094/1,82094,95
421737,84162_rev1_d,test suite lww in entbackup restore is also re...,Frederick Booker,2017-10-09 19:52:13.747000000,NaN,CBQE-1898 lww tests are covered in py-entbacku...,1,Frederick Booker,1,http://review.couchbase.org/#/c/84162/1,84162,61


--------------------------------------------------------END--------------------------------------------------------------------

# Readability 
The Flesch-Kincaid Grade Level

In [27]:
len(useful_data)

30101

In [28]:
useful_data['Flesch-Kincaid'] = 0

C:\Users\GNR\AppData\Local\Temp\ipykernel_588\436516747.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_data['Flesch-Kincaid'] = 0


In [29]:
import textstat

for n in range(len(useful_data)):
    useful_data['Flesch-Kincaid'].iloc[n] = textstat.flesch_reading_ease(useful_data['Comment'].iloc[n])
#textstat.flesch_reading_ease(text)

C:\Users\GNR\AppData\Local\Temp\ipykernel_588\1877658995.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  useful_data['Flesch-Kincaid'].iloc[n] = textstat.flesch_reading_ease(useful_data['Comment'].iloc[n])


In [32]:
useful_data

,PR_number,Comment,Author,Date,Commit_message,Descriptions,status,main_author,revision_number,url,PR_number_no_patch_number,comment_length,Flesch-Kincaid
4,12050_rev2_d,I would prefer that you didn't submit thisI w...,Kenny Maynard,2012-01-04 22:01:33.920000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,318,77.27
5,12050_rev2_d,You also need to fix the commit message so it ...,Kenny Maynard,2012-01-04 22:02:19.848000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,77,80.62
9,12050_rev2_d,I would prefer that you didn't submit this-1 ...,Kenny Maynard,2012-01-05 22:42:21.672000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,76,66.74
10,12050_rev2_d,"I don't have a huge opinion on it, really. My...",Noah Dean,2012-01-06 03:50:33.697000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,2,http://review.couchbase.org/#/c/12050/2,12050,388,70.33
12,12050_rev3_d,"I think Rags is working on this today too, so ...",Andreas Short,2012-01-08 21:36:57.811000000,Refactoring changes summary:Membase has been r...,Potential Refactoring of Java Client to couchb...,1,Emerson Nolan,3,http://review.couchbase.org/#/c/12050/3,12050,161,73.17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
420134,80604_rev2_d,Got it - will remember that.,Ellie Kidd,2017-07-13 18:41:21.038000000,NaN,CBQE-3067: Use better FTS level compaction con...,1,Ellie Kidd,2,http://review.couchbase.org/#/c/80604/2,80604,28,83.32
420185,80992_rev1_d,MemoryQuota may set in param with default valu...,Frederick Booker,2017-07-18 15:43:30.472000000,NaN,CBQE-4209: Added the cbas installation and ser...,1,Alfred Park,1,http://review.couchbase.org/#/c/80992/1,80992,113,94.15
420805,82094_rev1_d,This change has already been reviewed by dev f...,Monique Barker,2017-08-09 02:25:02.503000000,NaN,Adding subquery tests to watson branch.,1,Dylan Paul,1,http://review.couchbase.org/#/c/82094/1,82094,95,72.32
421737,84162_rev1_d,test suite lww in entbackup restore is also re...,Frederick Booker,2017-10-09 19:52:13.747000000,NaN,CBQE-1898 lww tests are covered in py-entbacku...,1,Frederick Booker,1,http://review.couchbase.org/#/c/84162/1,84162,61,68.77


In [33]:
compression_opts = dict(method='zip',
                        archive_name='all_data_for_analysis.csv') 
useful_data.to_csv('all_data_for_analysis.zip', index=False, compression=compression_opts)